# Proyecto SARIMA - Análisis de Serie de Tiempo

Este notebook tiene como objetivo realizar el análisis y modelado de una serie de tiempo utilizando un modelo **SARIMA**.
Las secciones están predefinidas para facilitar el trabajo colaborativo durante el examen.
---

## 1. Importación de librerías
En esta sección se importarán todas las librerías necesarias para el proyecto.

In [10]:
# Ejemplo de importaciones
import requests
import plotly.express as px
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.tsa.statespace.sarimax import SARIMAX
import warnings
warnings.filterwarnings('ignore')

## 2. Descarga y carga de la serie de tiempo
En esta sección se descargará la serie de tiempo desde la fuente correspondiente y se visualizará su estructura inicial.

Get from API

In [11]:
'''
# Leer el token desde el archivo .secrets
with open(".secrets", "r") as f:
    token = f.read().strip()

'''

'\n# Leer el token desde el archivo .secrets\nwith open(".secrets", "r") as f:\n    token = f.read().strip()\n\n'

In [12]:
'''

# Endpoint para la serie SF43718 (Tipo de cambio FIX)
url = "https://www.banxico.org.mx/SieAPIRest/service/v1/series/SF43718/datos"

# Encabezados con el token
headers = {
    "Bmx-Token": token
}

# Hacemos la petición
response = requests.get(url, headers=headers)

if response.status_code == 200:
    data = response.json()
    serie = data["bmx"]["series"][0]["datos"]

    # Convertir a DataFrame
    df = pd.DataFrame(serie)
    df.rename(columns={"fecha": "Fecha", "dato": "TipoCambio"}, inplace=True)

    # Convertir columnas
    df["Fecha"] = pd.to_datetime(df["Fecha"], format="%d/%m/%Y")
    df["TipoCambio"] = pd.to_numeric(df["TipoCambio"], errors="coerce")

    # Crear carpeta data si no existe
    os.makedirs("data", exist_ok=True)

    # Guardar en CSV
    csv_path = "data/tipo_cambio_fix.csv"
    df.to_csv(csv_path, index=False, encoding="utf-8-sig")
    print(f"Datos guardados en {csv_path}")

    # Graficar con Plotly
    fig = px.line(df, x="Fecha", y="TipoCambio",
                  title="Tipo de cambio FIX (Peso/Dólar) - Banxico",
                  labels={"TipoCambio": "Pesos por USD"})
    fig.show()

else:
    print("Error en la consulta:", response.status_code, response.text)

'''

'\n\n# Endpoint para la serie SF43718 (Tipo de cambio FIX)\nurl = "https://www.banxico.org.mx/SieAPIRest/service/v1/series/SF43718/datos"\n\n# Encabezados con el token\nheaders = {\n    "Bmx-Token": token\n}\n\n# Hacemos la petición\nresponse = requests.get(url, headers=headers)\n\nif response.status_code == 200:\n    data = response.json()\n    serie = data["bmx"]["series"][0]["datos"]\n\n    # Convertir a DataFrame\n    df = pd.DataFrame(serie)\n    df.rename(columns={"fecha": "Fecha", "dato": "TipoCambio"}, inplace=True)\n\n    # Convertir columnas\n    df["Fecha"] = pd.to_datetime(df["Fecha"], format="%d/%m/%Y")\n    df["TipoCambio"] = pd.to_numeric(df["TipoCambio"], errors="coerce")\n\n    # Crear carpeta data si no existe\n    os.makedirs("data", exist_ok=True)\n\n    # Guardar en CSV\n    csv_path = "data/tipo_cambio_fix.csv"\n    df.to_csv(csv_path, index=False, encoding="utf-8-sig")\n    print(f"Datos guardados en {csv_path}")\n\n    # Graficar con Plotly\n    fig = px.line(df

In [27]:
df = pd.read_csv(r'data\tipo_cambio_fix.csv')
df.set_index('Fecha', inplace=True)
df.head()

,TipoCambio
Fecha,
1991-11-12,3.0735
1991-11-13,3.0712
1991-11-14,3.0718
1991-11-15,3.0684
1991-11-18,3.0673


In [28]:
# Graficar con Plotly
fig = px.line(
    df,
    x=df.index,
    y="TipoCambio",
    title="Tipo de cambio FIX (Peso/Dólar) - Banxico",
    labels={"TipoCambio": "Pesos por USD"}
)

# Mostrar gráfico interactivo
fig.show()

In [29]:
import pandas as pd
from datetime import datetime

fecha_inicio = datetime(2021, 1, 1)
hoy = datetime.today()

dias = (hoy - fecha_inicio).days
print(f"Días entre 2021-01-01 y hoy: {dias}")

# Asegúrate de que el índice sea datetime
df.index = pd.to_datetime(df.index)

# Filtrar desde 2021-01-01 hasta hoy
fecha_inicio = "2021-01-01"
hoy = pd.Timestamp.today()

df = df.loc[fecha_inicio:hoy]

# Graficar con Plotly
fig = px.line(
    df,
    x=df.index,
    y="TipoCambio",
    title="Tipo de cambio FIX (Peso/Dólar) - Banxico",
    labels={"TipoCambio": "Pesos por USD"}
)

# Mostrar gráfico interactivo
fig.show()

Días entre 2021-01-01 y hoy: 1720


## 3. Division Train/Test

In [30]:
df_train = df[:-12]
df_test = df[-12:]

display(df_train)
display(df_test)

,TipoCambio
Fecha,
2021-01-04,19.8457
2021-01-05,19.9437
2021-01-06,19.7250
2021-01-07,19.9100
2021-01-08,19.9705
...,...
2025-08-25,18.6368
2025-08-26,18.6843
2025-08-27,18.6912


,TipoCambio
Fecha,
2025-09-01,18.6460
2025-09-02,18.7072
2025-09-03,18.6873
2025-09-04,18.7577
2025-09-05,18.6792
2025-09-08,18.6550
2025-09-09,18.6353
2025-09-10,18.5920
2025-09-11,18.5287


## 4. Pruebas de estacionariedad
Esta sección incluye funciones para verificar si la serie es estacionaria.

**Pruebas incluidas:**
- Prueba ADF (Augmented Dickey-Fuller)
- Prueba KPSS (Kwiatkowski–Phillips–Schmidt–Shin)

La función devolverá un diagnóstico con base en los valores p.

In [33]:
def adf_test(series):
    result = adfuller(series)
    p_value = result[1]
    print(f"ADF Statistic: {result[0]:.4f}, p-value: {p_value:.4f}")
    return p_value < 0.05  # Si es menor a 0.05, la serie es estacionaria

def kpss_test(series):
    result = kpss(series, regression='c')
    p_value = result[1]
    print(f"KPSS Statistic: {result[0]:.4f}, p-value: {p_value:.4f}")
    return p_value > 0.05  # Si es mayor a 0.05, la serie es estacionaria

def check_stationarity(series):
    print('--- Prueba ADF ---')
    adf_result = adf_test(series)
    print('\n--- Prueba KPSS ---')
    kpss_result = kpss_test(series)

    if adf_result and kpss_result:
        print('\nConclusión: La serie es estacionaria.')
        return True
    else:
        print('\nConclusión: La serie NO es estacionaria.')
        return False

In [32]:
# check_stationarity(df)

## 5. Auto-diferenciación
En esta sección se aplicarán diferenciaciones automáticas hasta lograr que la serie sea estacionaria.

In [ ]:
def auto_difference(series, max_diff=5):
    diff_count = 0
    temp_series = series.copy()

    while not check_stationarity(temp_series) and diff_count < max_diff:
        temp_series = temp_series.diff().dropna()
        diff_count += 1
        print(f"Aplicando diferenciación #{diff_count}")

    print(f"Número total de diferenciaciones aplicadas: {diff_count}")
    return temp_series, diff_count

In [ ]:
df_diff, count = auto_difference(df)

--- Prueba ADF ---
ADF Statistic: -1.5640, p-value: 0.5017

--- Prueba KPSS ---
KPSS Statistic: 13.6168, p-value: 0.0100

Conclusión: La serie NO es estacionaria.
Aplicando diferenciación #1
--- Prueba ADF ---
ADF Statistic: -14.1279, p-value: 0.0000

--- Prueba KPSS ---
KPSS Statistic: 0.0671, p-value: 0.1000

Conclusión: La serie es estacionaria.
Número total de diferenciaciones aplicadas: 1


## 6. Modelado SARIMA
Aquí se construirá y entrenará el modelo SARIMA utilizando la serie estacionaria.

In [ ]:
# TODO: Definir y ajustar el modelo SARIMA
# model = SARIMAX(serie_estacionaria, order=(p,d,q), seasonal_order=(P,D,Q,m))
# results = model.fit()
# results.summary()

## 7. Metricas

## 8. Conclusiones
En esta sección se anotarán las conclusiones del análisis y los resultados del modelo.